In [1]:
---
title: ELN 
author: Daniella Prado 
date: 'June 11, 2024'
execute:
  echo: false
format: 
    html: 
        code-fold: True
    code: True 
    
---

SyntaxError: invalid syntax (114929754.py, line 1)

In [2]:
#| echo: false
# Imports 
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import date
import base64
from datetime import date


In [2]:
'''{python}
#| echo: false
def get_base64_image(image_path):
    with open(image_path, 'rb') as f:
        image_bytes = f.read()
    encoded = base64.b64encode(image_bytes).decode('utf-8')
    return f"<img src='data:image/jpeg;base64,{encoded}' width='120'>"
# Setup styling
display(HTML("""
<style>
    .container { width:100% !important; }
    .widget-button { background-color: #004080 !important; color: white; border-radius: 5px; }
    .main-box { padding: 20px; background-color: #fafafa; border-left: 1px solid #ccc; }
    .sidebar-box { background-color: #f0f0f0; padding: 20px; border-right: 2px solid #ccc; height: 100%; }
    .footer { text-align: center; color: #888; margin-top: 40px; font-size: 13px; }
</style>
"""))

# Global store for experiments
experiment_data = []
archived_data = []

# Output container
main_output = widgets.Output()

# make sure text is between 150 - 200 words 
def make_limited_textarea(label, default_val):
    textarea = widgets.Textarea(
        description='',
        value=default_val,
        layout=widgets.Layout(width="100%", height="80px"),
        placeholder="150–200 characters"
    )
    counter = widgets.Label(f"{len(textarea.value)} / 200")
    # start counting to make sure text doesnt go over 
    def on_text_change(change):
        val = change['new']
        if len(val) > 200:
            textarea.value = val[:200]
        counter.value = f"{len(textarea.value)} / 200"

    textarea.observe(on_text_change, names='value')
    return widgets.VBox([textarea, counter])

# to view notebook, (first time AND editing) 
def show_add_experiment_form(existing=None):
    with main_output:
        clear_output()
        is_editing = existing is not None

        display(widgets.HTML(f"<h2>{'Edit' if is_editing else 'New'} Lab Notebook Entry</h2>"))

        # Title and Meta
        title = widgets.Text(value=existing['title'] if existing else '', placeholder="150–200 characters")
        owner = widgets.Text(value=existing['owner'] if existing else '', placeholder="150–200 characters")
        start_date = widgets.DatePicker(value=existing['start_date'] if existing else date.today())
        end_date = widgets.DatePicker(value=existing['end_date'] if existing else None)

        # Sections with live counters
        hypothesis_box = make_limited_textarea("Hypothesis", existing['hypothesis'] if existing else '')
        materials_box = make_limited_textarea("Materials & Methods", existing['materials'] if existing else '')
        # make materials rows and when i add a row it goes to a database about materials used 
        observations_box = make_limited_textarea("Observations", existing['observations'] if existing else '')
        conclusions_box = make_limited_textarea("Conclusions", existing['conclusions'] if existing else '')
        # ability to display molecular reactions and modules 
        # compartemantelization per scientist, sometimes scientists can;t look at others' work 
        # ability to search across data/mine data - make notebooks private/public 
        submit_btn = widgets.Button(description="Save Entry", button_style='success')
        # information saved 
        def on_submit(b):
            with main_output:
                clear_output()
                data = {
                    'title': title.value[:200],
                    'owner': owner.value[:200],
                    'start_date': start_date.value,
                    'end_date': end_date.value,
                    'hypothesis': hypothesis_box.children[0].value[:200],
                    'materials': materials_box.children[0].value[:200],
                    'observations': observations_box.children[0].value[:200],
                    'conclusions': conclusions_box.children[0].value[:200],
                }

                if is_editing:
                    existing.update(data)
                    display(widgets.HTML(f"<h3>Updated Notebook Entry: <i>{title.value}</i></h3>"))
                else:
                    experiment_data.append(data)
                    display(widgets.HTML(f"<h3>Saved Notebook Entry: <i>{title.value}</i></h3>"))

        submit_btn.on_click(on_submit)

        # Layout: notebook section headers
        notebook_layout = widgets.VBox([
            widgets.HTML("<h3>Title</h3>"), title,
            widgets.HTML("<h4>Scientist</h4>"), owner,
            widgets.HTML("<h4>Date Range</h4>"), widgets.HBox([start_date, end_date]),
            widgets.HTML("<h4>Hypothesis</h4>"), hypothesis_box,
            widgets.HTML("<h4>Materials & Methods</h4>"), materials_box,
            widgets.HTML("<h4>Observations</h4>"), observations_box,
            widgets.HTML("<h4>Conclusions</h4>"), conclusions_box,
            submit_btn
        ], layout=widgets.Layout(width='100%', padding='10px'))

        display(notebook_layout)
        
# Make sure people want to delete before 
def confirm_delete(index):
    with main_output:
        clear_output()
        exp = experiment_data[index]
        display(widgets.HTML(f"""
            <h3>Are you sure you want to delete <i>{exp['title']}</i>?</h3>
            <p style='color:red;'>This action cannot be undone.</p>
        """))

        yes_btn = widgets.Button(description="Yes, Delete", button_style='danger')
        no_btn = widgets.Button(description="Cancel", button_style='info')

        # Fix: Capture `index` with default arguments
        def perform_delete(b, index=index):
            deleted = experiment_data.pop(index)
            clear_output()
            display(widgets.HTML(f"<h3>🗑️ Deleted: <i>{deleted['title']}</i></h3>"))
            show_previous_experiments()
        # cancel delete, keep everything as is 
        def cancel_delete(b):
            show_previous_experiments()

        yes_btn.on_click(lambda b: perform_delete(b, index))
        no_btn.on_click(cancel_delete)

        btn_row = widgets.HBox([yes_btn, no_btn])
        display(btn_row)
        
# View Experiments
def show_previous_experiments():
    with main_output:
        clear_output()
        display(widgets.HTML("<h2>Previous Experiments</h2>"))

        if not experiment_data:
            display(widgets.HTML("<p><i>No experiments saved yet.</i></p>"))
        else:
            experiment_widgets = []

            for i, exp in enumerate(experiment_data):
                edit_btn = widgets.Button(
                    description=f"{exp['title']} ({exp['start_date']})",
                    button_style='info',
                    layout=widgets.Layout(width='auto')
                )
                archive_btn = widgets.Button(
                    description="Archive",
                    button_style='warning',
                    layout=widgets.Layout(width='100px')
                )
                
                # Need to capture index in default arguments
                def delete_experiment(btn, index=i):
                    with main_output:
                        clear_output()
                        deleted = experiment_data.pop(index)
                        display(widgets.HTML(f"<h3>Deleted: {deleted['title']}</h3>"))
                        show_previous_experiments()

                edit_btn.on_click(lambda b, e=exp: show_add_experiment_form(e))
                archive_btn.on_click(lambda b, idx=i: archive_experiment(idx))

                row = widgets.HBox([edit_btn, archive_btn], layout=widgets.Layout(justify_content='space-between'))
                experiment_widgets.append(row)

            display(widgets.VBox(experiment_widgets))
            
def archive_experiment(index):
    with main_output:
        clear_output()
        exp = experiment_data.pop(index)
        archived_data.append(exp)
        display(widgets.HTML(f"<h3>Archived: <i>{exp['title']}</i></h3>"))
        show_previous_experiments()
def show_archive():
    with main_output:
        clear_output()
        display(widgets.HTML("<h2>Archived Experiments</h2>"))

        if not archived_data:
            display(widgets.HTML("<p><i>No archived experiments.</i></p>"))
        else:
            archive_widgets = []

            for i, exp in enumerate(archived_data):
                restore_btn = widgets.Button(
                    description=f"Restore: {exp['title']} ({exp['start_date']})",
                    button_style='success',
                    layout=widgets.Layout(width='auto')
                )

                def restore(index=i):
                    experiment_data.append(archived_data.pop(index))
                    show_archives()

                restore_btn.on_click(lambda b, idx=i: restore(idx))
                archive_widgets.append(restore_btn)

            display(widgets.VBox(archive_widgets))

# Sidebar buttons
add_exp_btn = widgets.Button(description="Add Experiment", layout=widgets.Layout(width='100%'))
view_exp_btn = widgets.Button(description="View Experiments", layout=widgets.Layout(width='100%'))
archive_btn = widgets.Button(description="Archives", layout=widgets.Layout(width='100%'))

add_exp_btn.on_click(lambda b: show_add_experiment_form())
view_exp_btn.on_click(lambda b: show_previous_experiments())
archive_btn.on_click(lambda b: show_archive())

# Sidebar
logo_html = widgets.HTML(get_base64_image("20_15_visioneers_logo.jpg"))
sidebar = widgets.VBox([
    widgets.HTML("<h2> Prototype ELN</h2>"),
    widgets.Label(f"Date: {date.today()}"),
    widgets.HTML("<hr>"),
    add_exp_btn,
    view_exp_btn,
    archive_btn,
    logo_html,
    widgets.HTML("<div class='footer'>20/15 Visioneers<br>ELN demo</div>")
], layout=widgets.Layout(
    width='25%',
    border='solid 1px lightgray',
    padding='10px',
    height='100vh'  
), _dom_classes=['sidebar-box'])

# Layout
app_layout = widgets.HBox([
    sidebar,
    widgets.VBox([main_output], layout=widgets.Layout(padding='20px'))
], layout=widgets.Layout(border='solid 1px #ddd'))

# Display UI
display(app_layout)

# Default View
with main_output:
    clear_output()
    display(widgets.HTML("<h2>Welcome to your ELN </h2><p>Click an option on the left to get started.</p>"))
'''

'{python}\n#| echo: false\ndef get_base64_image(image_path):\n    with open(image_path, \'rb\') as f:\n        image_bytes = f.read()\n    encoded = base64.b64encode(image_bytes).decode(\'utf-8\')\n    return f"<img src=\'data:image/jpeg;base64,{encoded}\' width=\'120\'>"\n# Setup styling\ndisplay(HTML("""\n<style>\n    .container { width:100% !important; }\n    .widget-button { background-color: #004080 !important; color: white; border-radius: 5px; }\n    .main-box { padding: 20px; background-color: #fafafa; border-left: 1px solid #ccc; }\n    .sidebar-box { background-color: #f0f0f0; padding: 20px; border-right: 2px solid #ccc; height: 100%; }\n    .footer { text-align: center; color: #888; margin-top: 40px; font-size: 13px; }\n</style>\n"""))\n\n# Global store for experiments\nexperiment_data = []\narchived_data = []\n\n# Output container\nmain_output = widgets.Output()\n\n# make sure text is between 150 - 200 words \ndef make_limited_textarea(label, default_val):\n    textarea = wi